<a href="https://colab.research.google.com/github/karenbennis/Xy/blob/ml_model/ml_model_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <br>**Connect to Database**<br><br>

In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

#Interact with SQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Start Spark Session(Creating spark application with name defined by appName()) ---IMPORTED WITH EVERY COLAB NOTEBOOK
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("database_transformation").config("spark.driver.memory","5g").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


--2020-07-26 16:19:55--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.55MB/s    in 0.2s    

2020-07-26 16:19:56 (3.55 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# gcloud login and check the DB
!gcloud auth login
!gcloud config set project 'xy-yelp'
!gcloud sql instances describe 'xy-yelp'

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=TGHQzzth5e5AMSwqAw6CpiH3wec9voSu-hui-MK-he0&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/2QHe7YUZLypxZoQQ6PhcY2igPX1F0aY8znLdi8Pv6YIxOPyVrLPM_vU

You are now logged in as [helenly25@gmail.com].
Your current project is [xy-yelp].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
backendType: SECOND_GEN
connectionName: xy-yelp:northamerica-northeast1:xy-yelp
databaseV

In [ ]:
# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy
# "connectionName" is from the previous block
!nohup ./cloud_sql_proxy -instances="xy-yelp:northamerica-northeast1:xy-yelp"=tcp:5432 &
!sleep 30s

cloud_sql_proxy: Text file busy
nohup: appending output to 'nohup.out'


In [ ]:
db_password = 'kjhbyelpdb'

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://127.0.0.1:5432/xy_yelp_db"
config = {"user":"postgres", 
          "password": db_password, 
          "driver":"org.postgresql.Driver"}

## **Extract tables**

In [2]:
# Load in data
import pandas as pd
pandas_df = pd.read_csv("https://raw.githubusercontent.com/karenbennis/Xy/storyboard/uniform_yelp.csv")

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0


## **Transformation**

In [21]:
# Create function for binary classification
def good(x):
    if x > 3:
        return 1
    if x < 4:
        return 0

# Create the class column splitting review into positive and negative
pandas_df['label']=pandas_df['stars'].apply(good)
pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
3,bZ02moAXlosgWPM3pXSHWw,zE49S2Em3l7vgIlvFzZFOw,NF6di6YcQxN0rDAleE7SyQ,3,2014-12-30,Today was the first time I sat down a table. I...,0,0,0,0,Today was the first time I sat down a table I ...,"[today, was, the, first, time, i, sat, down, a...","[today, first, time, sat, table, like, wait, w...","[today, first, time, sat, tabl, like, wait, we...","[today, first, time, sat, table, like, wait, w...",1682,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0


In [22]:
# Remove 3 stars
pandas_df = pandas_df.loc[pandas_df['stars'] != 3]
pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0
5,Hbn8Wn1ryc_SloxZ7jRGWA,st8UrI7Qkwi3nvHH0zpqUg,sFg3G8-oMGeiHYsVqKN7PQ,2,2017-06-27,Granted it's a possibility that this restauran...,0,0,0,0,Granted its a possibility that this restaurant...,"[granted, its, a, possibility, that, this, res...","[granted, possibility, restaurant, mom, pop, t...","[grant, possibl, restaur, mom, pop, type, plac...","[granted, possibility, restaurant, mom, pop, t...",1844,0


In [23]:
# Import dependencies for nltk
# https://towardsdatascience.com/natural-language-processing-nlp-for-machine-learning-d44498845d5b
import nltk

In [24]:
# Import string and punctuations
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [25]:
# Function to remove Punctuation
def remove_punct(text):

  # Discard all punctuations
  text_nopunct = ''.join([char for char in text if char not in string.punctuation])
  return text_nopunct

pandas_df['body_text_clean'] = pandas_df['text'].apply(lambda x: remove_punct(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0
5,Hbn8Wn1ryc_SloxZ7jRGWA,st8UrI7Qkwi3nvHH0zpqUg,sFg3G8-oMGeiHYsVqKN7PQ,2,2017-06-27,Granted it's a possibility that this restauran...,0,0,0,0,Granted its a possibility that this restaurant...,"[granted, its, a, possibility, that, this, res...","[granted, possibility, restaurant, mom, pop, t...","[grant, possibl, restaur, mom, pop, type, plac...","[granted, possibility, restaurant, mom, pop, t...",1844,0


In [26]:
# Import re
import re

# Function to Tokenize words
def tokenize(text):

  # W+ means that either a word character (A-Za-z0-9) or a dash (-) can go there
  tokens = re.split('\W+', text)
  return tokens

# Convert to lowercase as Python is case-sensitive
pandas_df['body_text_tokenized'] = pandas_df['body_text_clean'].apply(lambda x: tokenize(x.lower()))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0
5,Hbn8Wn1ryc_SloxZ7jRGWA,st8UrI7Qkwi3nvHH0zpqUg,sFg3G8-oMGeiHYsVqKN7PQ,2,2017-06-27,Granted it's a possibility that this restauran...,0,0,0,0,Granted its a possibility that this restaurant...,"[granted, its, a, possibility, that, this, res...","[granted, possibility, restaurant, mom, pop, t...","[grant, possibl, restaur, mom, pop, type, plac...","[granted, possibility, restaurant, mom, pop, t...",1844,0


In [27]:
# Remove all English stopwords
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# Function to remove stopwords
def remove_stopwords(tokenized_list):

  # Remove all stopwords
  text = [word for word in tokenized_list if word not in stopword]
  return text

pandas_df['body_text_nostop'] = pandas_df['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0
5,Hbn8Wn1ryc_SloxZ7jRGWA,st8UrI7Qkwi3nvHH0zpqUg,sFg3G8-oMGeiHYsVqKN7PQ,2,2017-06-27,Granted it's a possibility that this restauran...,0,0,0,0,Granted its a possibility that this restaurant...,"[granted, its, a, possibility, that, this, res...","[granted, possibility, restaurant, mom, pop, t...","[grant, possibl, restaur, mom, pop, type, plac...","[granted, possibility, restaurant, mom, pop, t...",1844,0


In [29]:
# Import PorterStemmer
from nltk.stem import PorterStemmer

# Create an instance for stemmer
ps = nltk.PorterStemmer()

# Function for stemming
def stemming(tokenized_text):

  text = [ps.stem(word) for word in tokenized_text]
  return text

pandas_df['body_text_stemmed'] = pandas_df['body_text_nostop'].apply(lambda x: stemming(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0
5,Hbn8Wn1ryc_SloxZ7jRGWA,st8UrI7Qkwi3nvHH0zpqUg,sFg3G8-oMGeiHYsVqKN7PQ,2,2017-06-27,Granted it's a possibility that this restauran...,0,0,0,0,Granted its a possibility that this restaurant...,"[granted, its, a, possibility, that, this, res...","[granted, possibility, restaurant, mom, pop, t...","[grant, possibl, restaur, mom, pop, type, plac...","[granted, possibility, restaurant, mom, pop, t...",1844,0


In [30]:
# Download and import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

# Create an instance
wn = nltk.WordNetLemmatizer()

# Function for Lemmatization
def lemmatizing(tokenized_text):

  text = [wn.lemmatize(word) for word in tokenized_text]
  return text

pandas_df['body_text_lemmatized'] = pandas_df['body_text_nostop'].apply(lambda x: lemmatizing(x))

pandas_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0
5,Hbn8Wn1ryc_SloxZ7jRGWA,st8UrI7Qkwi3nvHH0zpqUg,sFg3G8-oMGeiHYsVqKN7PQ,2,2017-06-27,Granted it's a possibility that this restauran...,0,0,0,0,Granted its a possibility that this restaurant...,"[granted, its, a, possibility, that, this, res...","[granted, possibility, restaurant, mom, pop, t...","[grant, possibl, restaur, mom, pop, type, plac...","[granted, possibility, restaurant, mom, pop, t...",1844,0


In [31]:
# Add a length column to dataset
pandas_df['length'] = pandas_df['text'].apply(len)
pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,class,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length,label
0,f5pGCpvkpRpJixZ0zA3hCg,8p2nss7UoZmIVZTr1IjR3w,caWUE0ItqsG51OaBVlr4Eg,2,2016-11-13,We went here three more times for lunch and tw...,0,0,0,0,We went here three more times for lunch and tw...,"[we, went, here, three, more, times, for, lunc...","[went, three, times, lunch, twice, dinner, thi...","[went, three, time, lunch, twice, dinner, thin...","[went, three, time, lunch, twice, dinner, thin...",572,0
1,8lPKsNFBiLmVL5nbsUXaZw,O3pSxv1SyHpY4qi4Q16KzA,dc3uoAmNo5STqKV6mlD_aA,1,2017-05-30,My husband and I went to the Drake for lunch t...,2,1,1,0,My husband and I went to the Drake for lunch t...,"[my, husband, and, i, went, to, the, drake, fo...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...","[husband, went, drake, lunch, today, sat, pati...",407,0
2,CANhCLzOoZ0mkL3mpnUSNg,ffC9zmbY4pBOS9ByrWoXxQ,sR9hPrIaG-J-GLcl4yaiLw,1,2017-11-19,"Very Problematic\nI'm gay, I'm not ashamed to ...",0,0,0,0,Very Problematic\nIm gay Im not ashamed to say...,"[very, problematic, im, gay, im, not, ashamed,...","[problematic, im, gay, im, ashamed, say, felt,...","[problemat, im, gay, im, asham, say, felt, ash...","[problematic, im, gay, im, ashamed, say, felt,...",483,0
4,TmIla5Eh5SSLJ_bKgH4Syg,xde2rO3XVt0Do8kLRIt2Dw,Wc9UpJhOcdSj7olZkz7SJA,2,2013-02-19,I ordered chicken tacos with no cheese to go. ...,4,1,0,0,I ordered chicken tacos with no cheese to go ...,"[i, ordered, chicken, tacos, with, no, cheese,...","[ordered, chicken, tacos, cheese, go, got, off...","[order, chicken, taco, chees, go, got, offic, ...","[ordered, chicken, taco, cheese, go, got, offi...",324,0
5,Hbn8Wn1ryc_SloxZ7jRGWA,st8UrI7Qkwi3nvHH0zpqUg,sFg3G8-oMGeiHYsVqKN7PQ,2,2017-06-27,Granted it's a possibility that this restauran...,0,0,0,0,Granted its a possibility that this restaurant...,"[granted, its, a, possibility, that, this, res...","[granted, possibility, restaurant, mom, pop, t...","[grant, possibl, restaur, mom, pop, type, plac...","[granted, possibility, restaurant, mom, pop, t...",1844,0


## <br></br>**Pipeline**<br></br>

In [40]:
# Make a copy of the data
pandas_df_copy = pandas_df.copy()

# Select columns for the DataFrame
pandas_df_copy = pandas_df_copy[['review_id', 'text', 'stars', 'cool', 'useful', 'funny', 'date', 'business_id', 'user_id', 'length', 'body_text_nostop', 'body_text_stemmed', 'body_text_lemmatized','label']]

# Convert pandas_df to sparks df
spark_df = spark.createDataFrame(pandas_df_copy)
spark_df.show(5)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+-----+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|label|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+-----+
|f5pGCpvkpRpJixZ0z...|We went here thre...|    2|   0|     0|    0|2016-11-13|caWUE0ItqsG51OaBV...|8p2nss7UoZmIVZTr1...|   572|[went, three, tim...|[went, three, tim...|[went, three, tim...|    0|
|8lPKsNFBiLmVL5nbs...|My husband and I ...|    1|   1|     2|    1|2017-05-30|dc3uoAmNo5STqKV6m...|O3pSxv1SyHpY4qi4Q...|   407|[husband, went, d...|[husband, went, d...|[husband, went, d...|    0|
|CANhCLzOoZ0mkL

In [41]:
# Import functions
from pyspark.ml.feature import HashingTF, IDF

In [ ]:
# Make stars values a list
# from pyspark.sql.functions import col, split
# spark_df = spark_df.withColumn("star_array", split(col("stars"), " "))
# spark_df.show()

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|f5pGCpvkpRpJixZ0z...|We went here thre...|    2|   0|     0|    0|2016-11-13|caWUE0ItqsG51OaBV...|8p2nss7UoZmIVZTr1...|   572|[went, three, tim...|[went, three, tim...|[went, three, tim...|       [2]|
|8lPKsNFBiLmVL5nbs...|My husband and I ...|    1|   1|     2|    1|2017-05-30|dc3uoAmNo5STqKV6m...|O3pSxv1SyHpY4qi4Q...|   407|[husband, went, d...|[husband, went, d...|[husband, went, d...|  

In [ ]:
# # Initialize a CoutVectorizer
# from pyspark.ml.feature import CountVectorizer
# star_vectorizer = CountVectorizer(inputCol="star_array", outputCol="stars_one_hot", vocabSize=5, minDF=1.0)

In [ ]:
# Create a vector model
# star_vector_model = star_vectorizer.fit(spark_df)

In [ ]:
# # One hot encoded column
# df_ohe = star_vector_model.transform(spark_df)
# df_ohe.show(3)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|stars_one_hot|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|f5pGCpvkpRpJixZ0z...|We went here thre...|    2|   0|     0|    0|2016-11-13|caWUE0ItqsG51OaBV...|8p2nss7UoZmIVZTr1...|   572|[went, three, tim...|[went, three, tim...|[went, three, tim...|       [2]|(5,[4],[1.0])|
|8lPKsNFBiLmVL5nbs...|My husband and I ...|    1|   1|     2|    1|2017-05-30|dc3uoAmNo5STqKV6m...|O3pSxv1SyHpY4qi4Q...|   407|[husband,

In [42]:
# Create all the features to the data set
hashingTF = HashingTF(inputCol="body_text_stemmed", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [43]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vector 
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [44]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[hashingTF, idf, clean_up])

In [45]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(spark_df)
cleaned = cleaner.transform(spark_df)

In [46]:
cleaned.show(5)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|label|          hash_token|           idf_token|            features|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|f5pGCpvkpRpJixZ0z...|We went here thre...|    2|   0|     0|    0|2016-11-13|caWUE0ItqsG51OaBV...|8p2nss7UoZmIVZTr1...|   572|[went, three, tim...|[went, three, tim...|[went, three, tim...|    0|(262144,[4200,299...|(26

# **Machine Learning Models**

In [48]:
#Drop intermediate columns
x=cleaned.select('features', 'label')
x.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[4200,299...|    0|
|(262145,[6872,156...|    0|
|(262145,[3067,690...|    0|
|(262145,[27564,31...|    0|
|(262145,[976,2437...|    0|
|(262145,[22567,29...|    1|
|(262145,[6317,880...|    0|
|(262145,[976,1353...|    0|
|(262145,[6908,109...|    0|
|(262145,[6908,826...|    1|
|(262145,[1353,181...|    0|
|(262145,[11137,13...|    0|
|(262145,[4200,538...|    0|
|(262145,[32699,61...|    1|
|(262145,[6317,223...|    0|
|(262145,[1353,218...|    1|
|(262145,[34343,44...|    0|
|(262145,[2325,517...|    0|
|(262145,[12336,21...|    1|
|(262145,[1353,294...|    0|
+--------------------+-----+
only showing top 20 rows



In [49]:
# Check dtypes 
x.dtypes

[('features', 'vector'), ('label', 'bigint')]

In [ ]:
# Import col
# from pyspark.sql.functions import col

# # Change column DataType for stars
# x = x.withColumn('label', col('label').cast('int'))

# x.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[4200,299...|    4|
|(262145,[6872,156...|    3|
|(262145,[3067,690...|    3|
|(262145,[1353,232...|    1|
|(262145,[27564,31...|    4|
+--------------------+-----+
only showing top 5 rows



**Naive Bayes**

In [50]:
# Break data down into a training set and a testing set
training, testing = x.randomSplit([0.8, 0.2], 21)

In [51]:
from pyspark.ml.classification import NaiveBayes
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [52]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.738891


**Logistic Regression**

In [53]:
# Break data down into a training set and a testing set
training, testing = x.randomSplit([0.8, 0.2], 21)

In [54]:
from pyspark.ml.classification import LogisticRegression
# Create a Naive Bayes model and fit training data
lg = LogisticRegression()
predictor = lg.fit(training)

In [55]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.832530


**Multilayer Percepron**

In [56]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [57]:
# Split the data into train and test
splits = x.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

In [58]:
train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[47,34343...|    1|
|(262145,[57,7324,...|    1|
|(262145,[97,1353,...|    0|
|(262145,[97,3831,...|    1|
|(262145,[97,4402,...|    0|
|(262145,[97,13957...|    1|
|(262145,[98,6646,...|    0|
|(262145,[170,427,...|    0|
|(262145,[170,1353...|    0|
|(262145,[170,6243...|    1|
|(262145,[198,991,...|    1|
|(262145,[234,1176...|    1|
|(262145,[234,1353...|    0|
|(262145,[234,1288...|    0|
|(262145,[234,1395...|    0|
|(262145,[234,1729...|    1|
|(262145,[251,976,...|    0|
|(262145,[320,1076...|    1|
|(262145,[323,2544...|    1|
|(262145,[323,2742...|    1|
+--------------------+-----+
only showing top 20 rows



In [59]:
# specify layers for the neural network:
layers = [62000, 256, 5]

In [60]:
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234)

In [61]:
# train the model
model = trainer.fit(train)

Py4JJavaError: ignored